In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import scipy.stats as stats
import os

In [9]:
df = Path('Data/airline_2000_2005.csv')

df = pd.read_csv(df)

df.head()

,Unnamed: 0,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,...,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
0,0,53.0,32.0,-8.0,1642.0,1650,1545,65.0,NaN,0,...,10,NaN,DCA,NaN,N443US,7.0,14.0,US,NaN,2002
1,4,45.0,29.0,2.0,1120.0,1118,1030,48.0,NaN,0,...,6,0.0,CVG,0.0,N785CA,3.0,13.0,OH,0.0,2006
2,6,61.0,40.0,-3.0,1537.0,1540,1440,60.0,NaN,0,...,7,NaN,DFW,NaN,N286AE,7.0,14.0,MQ,NaN,2008
3,9,NaN,NaN,NaN,NaN,730,605,85.0,B,1,...,4,0.0,DEN,0.0,N705EV,0.0,0.0,EV,0.0,2005
4,13,100.0,81.0,-2.0,1220.0,1222,1045,97.0,NaN,0,...,11,0.0,GNV,0.0,N632AS,10.0,9.0,EV,0.0,2006


In [10]:
Path.cwd()

WindowsPath('c:/Users/purce/Documents/GitHub/module_7_project_1')

In [11]:
df.columns

Index(['Unnamed: 0', 'ActualElapsedTime', 'AirTime', 'ArrDelay', 'ArrTime',
       'CRSArrTime', 'CRSDepTime', 'CRSElapsedTime', 'CancellationCode',
       'Cancelled', 'CarrierDelay', 'DayOfWeek', 'DayofMonth', 'DepDelay',
       'DepTime', 'Dest', 'Distance', 'Diverted', 'FlightNum',
       'LateAircraftDelay', 'Month', 'NASDelay', 'Origin', 'SecurityDelay',
       'TailNum', 'TaxiIn', 'TaxiOut', 'UniqueCarrier', 'WeatherDelay',
       'Year'],
      dtype='object')

In [ ]:
jan_05_df = df[(df.Year ==2005) & (df.Month == 1)]
jan_05_df.info()


In [13]:
feb_05_df = df[(df.Year ==2005) & (df.Month == 2)]

594924

In [15]:
# loop throgh jan months and export to csv
for n in range(1,13):
    tmp_df = df[(df.Year ==2005) & (df.Month == n)]
    
    #export    
    file = f'Data/airlines_2005_{n}.csv'
    
    print(f'exporting file {file}')
    
    tmp_df.to_csv(file,encoding='utf-8',header=True)
    
    print("--------------------------")
    
    

exporting file Data/airlines_2005_1.csv
--------------------------
exporting file Data/airlines_2005_2.csv
--------------------------
exporting file Data/airlines_2005_3.csv
--------------------------
exporting file Data/airlines_2005_4.csv
--------------------------
exporting file Data/airlines_2005_5.csv
--------------------------
exporting file Data/airlines_2005_6.csv
--------------------------
exporting file Data/airlines_2005_7.csv
--------------------------
exporting file Data/airlines_2005_8.csv
--------------------------
exporting file Data/airlines_2005_9.csv
--------------------------
exporting file Data/airlines_2005_10.csv
--------------------------
exporting file Data/airlines_2005_11.csv
--------------------------
exporting file Data/airlines_2005_12.csv
--------------------------


In [16]:
tmp_df.head()

,Unnamed: 0,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,...,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
34,67,148.0,125.0,266.0,2156.0,1730,1402,148.0,NaN,0,...,12,0.0,MIA,0.0,N623AA,3.0,20.0,AA,0.0,2005
317,681,71.0,58.0,16.0,1931.0,1915,1800,75.0,NaN,0,...,12,0.0,STL,0.0,N784,3.0,10.0,WN,0.0,2005
876,1858,60.0,45.0,5.0,2210.0,2205,2105,60.0,NaN,0,...,12,0.0,LAX,0.0,N362,5.0,10.0,WN,0.0,2005
1065,2280,224.0,176.0,11.0,1212.0,1201,628,213.0,NaN,0,...,12,0.0,SJC,0.0,N532AA,32.0,16.0,AA,0.0,2005
1130,2415,60.0,41.0,24.0,1844.0,1820,1720,60.0,NaN,0,...,12,0.0,HOU,0.0,N509,10.0,9.0,WN,0.0,2005
